#LIBRARY IMPORT

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

#데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/LG AI대회/'

Mounted at /content/drive


In [5]:
#데이터 모두 불러옴

train_err  = pd.read_csv(PATH+'train_err_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')

#TRAIN 변수 만들기(DataFrame)

In [6]:
#error_type1만들기(일별 err_type별 도수)

#index = user_id , columns = errtype, value = 에러발생빈도수 인 피벗테이블 만들기

train_err['count'] = 1 #pivot테이블에서 계산을 위한 변수 1 추가
#피벗테이블 만들기
error_type1 = pd.pivot_table(train_err, values = 'count', index = 'user_id', columns = 'errtype',aggfunc = 'sum')
error_type1[29] = 0 #데이터에 29인 errtype이 없으므로 전부 0으로 하는 열 하나를 추가해줌
error_type1 = error_type1.reindex(columns = sorted(error_type1.columns)) #errtype을 오름차순으로 정리
error_type1.fillna(0, inplace=True) #na값은 0으로 바꿔줌

display(error_type1.head())
print(error_type1.shape)

errtype,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0


(15000, 42)


In [7]:
esum = []
#각 개체별 합을 esum이라는 변수로 만듦
for i in range(0,len(error_type1)):
  esum.append(error_type1.iloc[i].sum())
error_type1['esum'] = esum
error_type1.head()

errtype,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,esum
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317.0
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,2365.0
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,306.0
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,777.0


In [15]:
print(error_type1.shape)

(15000, 43)


#TEST변수 만들기(DataFrame)

In [8]:
#error_type2만들기(일별 err_type별 도수)
'''
#index = user_id , columns = errtype, value = 에러발생빈도수 인 피벗테이블 만들기

test_err['count'] = 1 #pivot테이블에서 계산을 위한 변수 1 추가
#피벗테이블 만들기
error_type2 = pd.pivot_table(test_err, values = 'count', index = 'user_id', columns = 'errtype',aggfunc = 'sum')
error_type2[29] = 0 #데이터에 29인 errtype이 없으므로 전부 0으로 하는 열 하나를 추가해줌
error_type2 = error_type2.reindex(columns = sorted(error_type2.columns)) #errtype을 오름차순으로 정리
error_type2.fillna(0, inplace=True) #na값은 0으로 바꿔줌
error_type2.tail()
'''

"\n#index = user_id , columns = errtype, value = 에러발생빈도수 인 피벗테이블 만들기\n\ntest_err['count'] = 1 #pivot테이블에서 계산을 위한 변수 1 추가\n#피벗테이블 만들기\nerror_type2 = pd.pivot_table(test_err, values = 'count', index = 'user_id', columns = 'errtype',aggfunc = 'sum')\nerror_type2[29] = 0 #데이터에 29인 errtype이 없으므로 전부 0으로 하는 열 하나를 추가해줌\nerror_type2 = error_type2.reindex(columns = sorted(error_type2.columns)) #errtype을 오름차순으로 정리\nerror_type2.fillna(0, inplace=True) #na값은 0으로 바꿔줌\nerror_type2.tail()\n"

In [9]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)
print(test_x)

100%|██████████| 16532648/16532648 [00:46<00:00, 351940.33it/s]

(14999, 42)
[[  0.   0.   0. ...  92.   0.   2.]
 [  0.   0.   3. ...   0.   0.   0.]
 [  0.   0.   0. ... 113.   1.   2.]
 ...
 [  0.   0.   0. ... 329.   2.   3.]
 [  0.   0.   0. ...  30.  49.   0.]
 [  0.   0.   0. ... 269.   0.   0.]]


In [13]:
test_x = pd.DataFrame(test_x)

esum = []
#각 개체별 합을 esum이라는 변수로 만듦
for i in range(0,len(test_x)):
  esum.append(test_x.iloc[i].sum())
test_x['esum'] = esum
test_x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,esum
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,2750.0
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.0
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,941.0
3,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,371.0
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,881.0


In [14]:
print(test_x.shape)

(14999, 43)
